In [2]:
import numpy as np
import matplotlib.pyplot as plt
import contourpy as cntr
# Use pyqt to enable interactive plotting
plt.switch_backend('Qt5Agg')
import numba as nb

In [3]:
def read_eqdsk(filename):
    '''! Read gEQDSK file

    @param filename Path to gEQDSK file
    @result Dictionary containing gEQDSK information
    '''
    def read_1d(fid, n):
        j = 0
        output = np.zeros((n,))
        for i in range(n):
            if j == 0:
                line = fid.readline()
            output[i] = line[j:j+16]
            j += 16
            if j == 16*5:
                j = 0
        return output

    def read_2d(fid, n, m):
        j = 0
        output = np.zeros((n, m))
        for k in range(n):
            for i in range(m):
                if j == 0:
                    line = fid.readline()
                output[k, i] = line[j:j+16]
                j += 16
                if j == 16*5:
                    j = 0
        return output
    # Read-in data
    eqdsk_obj = {}
    with open(filename, 'r') as fid:
        # Get sizes
        line = fid.readline()
        eqdsk_obj['case'] = line[:48]
        split_line = line[48:].split()
        eqdsk_obj['nr'] = int(split_line[-2])
        eqdsk_obj['nz'] = int(split_line[-1])
        # Read header content
        line_keys = [['rdim',  'zdim',  'rcentr',  'rleft',  'zmid'],
                     ['raxis', 'zaxis', 'psimag', 'psibry', 'bcentr'],
                     ['ip',    'skip',  'skip',   'skip',   'skip'],
                     ['skip',  'skip',  'skip',   'skip',   'skip']]
        for i in range(4):
            line = fid.readline()
            for j in range(5):
                if line_keys[i][j] == 'skip':
                    continue
                line_seg = line[j*16:(j+1)*16]
                eqdsk_obj[line_keys[i][j]] = float(line_seg)
        # Read flux profiles
        keys = ['fpol', 'pres', 'ffprim', 'pprime']
        for key in keys:
            eqdsk_obj[key] = read_1d(fid, eqdsk_obj['nr'])
        # Read PSI grid
        eqdsk_obj['psirz'] = read_2d(fid, eqdsk_obj['nz'],
                                        eqdsk_obj['nr'])
        # Read q-profile
        eqdsk_obj['qpsi'] = read_1d(fid, eqdsk_obj['nr'])
        # Read limiter count
        line = fid.readline()
        eqdsk_obj['nbbs'] = int(line.split()[0])
        eqdsk_obj['nlim'] = int(line.split()[1])
        # Read outer flux surface
        eqdsk_obj['rzout'] = read_2d(fid, eqdsk_obj['nbbs'], 2)
        # Read limiting corners
        eqdsk_obj['rzlim'] = read_2d(fid, eqdsk_obj['nlim'], 2)
    return eqdsk_obj


In [79]:


ptypes = [('R', nb.float64), 
          ('a', nb.float64),
          ('kappa', nb.float64),
          ('B0', nb.float64),
          ('Ip', nb.float64),
          ('q_a', nb.float64),
          ('H', nb.float64),
          ('M_i', nb.float64),
          ('fHe', nb.float64),
          ('f_LH', nb.float64),
          ('volgrid', nb.float64[:]),
          ('sqrtpsin', nb.float64[:]),
          ('impurityfractions', nb.float64[:])
          ]

impnames = np.array(["Helium", "Neon", "Argon", "Krypton", "Xenon", "Tungsten"], dtype=np.dtype('U10'))

@nb.experimental.jitclass(spec=ptypes)
class POPCON_params:
    """
    Physical parameters for the POPCON.
    """
    def __init__(self) -> None:
        self.R = -1.0
        "Major radius [m]"
        self.a = -1.0
        "Minor radius [m]"
        self.kappa = -1.0
        "Plasma elongation []"
        self.B0 = -1.0
        "Magnetic field at the magnetic axis [T]"
        self.Ip = -1.0
        "Plasma current [MA]"
        self.q_a = -1.0
        "Edge safety factor []"
        self.H = -1.0
        "Assumed H factor relative to chosen scaling law []"
        self.M_i = -1.0
        "Ion mass [AMU]"
        self.fHe = -1.0
        "Ash fraction []"
        self.f_LH = -1.0
        "Target LH fraction f_LH = P_sol / P_tot"
        self.volgrid = np.empty(0,dtype=np.float64)
        "Volume grid for flux surfaces"
        self.sqrtpsin = np.empty(1, dtype=np.float64)
        "Square root of poloidal flux ~ radial coordinate"
        # 0 = He, 1 = Ne, 2 = Ar, 3 = Kr, 4 = Xe, 5 = W
        self.impurityfractions = np.empty(6, dtype=np.float64)
        "Respective impurity fractions"
        pass

    def _aR(self,m:float) -> float:
        return m*self.a/self.R

In [80]:
params = POPCON_params()

In [72]:
impuritynames = np.zeros(1, dtype=str)


In [74]:
params._aR(1.1)

1.1

In [136]:
@nb.jit(nopython=True)
def example1(r:float):
    return r**2
@nb.jit(nopython=True)
def example2(r:float):
    return np.interp(r,np.linspace(0,1,10),np.linspace(0,1,10)**2)

ptypes = [('extprof', nb.boolean),
            ('defined', nb.boolean),
            ('alpha1', nb.float64),
            ('alpha2', nb.float64),
            ('offset', nb.float64),
            ('extprofr', nb.float64[:]),
            ('extprofvals', nb.float64[:])
          ]

@nb.experimental.jitclass(spec=ptypes)
class POPCON_profile:
    def __init__(self) -> None:
        self.extprof: bool
        self.defined: bool = False
        self.alpha1: float
        self.alpha2: float
        self.offset: float
        self.extprofr = np.empty(0,dtype=np.float64)
        self.extprofvals = np.empty(0,dtype=np.float64)

    def _addextprof(self, extprofr, extprofvals):
        if not self.defined:
            self.extprof = True
            self.defined = True
            self.extprofr = extprofr
            self.extprofvals = extprofvals
        else:
            raise SyntaxError("Profile already defined. Create a new object.")
        
    def _set_alpha_and_offset(self, alpha1, alpha2, offset):
        self.extprof = False
        self.defined = True
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.offset = offset
    
    def get_prof_val(self, r, v0):
        if self.extprof:
            return v0*np.interp(r,self.extprofr,self.extprofvals)
        else:
            return (v0-self.offset)*(1-r**self.alpha1)**self.alpha2+self.offset

In [137]:
p = profstest()
p._addextprof(np.linspace(0,1,10),np.linspace(0,1,10)**2)

In [141]:
p.get_prof_val(0.5,1)

0.25308641975308643